In [26]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
train_tape = "Z:/rppg/scamps_datatape_32x8x8_train.h5"
valid_tape = "Z:/rppg/scamps_datatape_32x8x8_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
train, valid = to_tf(train).cache(), to_tf(valid).cache()

In [56]:
model = keras.Sequential([
    layers.Input(shape=(32, 8, 8, 3)),
    layers.LayerNormalization(axis=(1,), trainable=False),
    layers.Conv3D(4, (2, 2, 2), (1, 2, 2), padding='same', activation='tanh'),
    layers.LayerNormalization(axis=(1,), trainable=False),
    layers.Conv3D(2, (2, 2, 2), (1, 2, 2), padding='same', activation='tanh'),
    layers.LayerNormalization(axis=(1,), trainable=False),
    layers.AvgPool3D((1, 2, 2)),
    layers.Conv3D(1, 1, 1),
    layers.Flatten(),
], name='NoobHeart')

model.compile(optimizer='adam', loss='mae')
print(f'Flops per frame:{get_flops(model, input_sig=[tf.TensorSpec([1, 32, 8, 8, 3])])/32:.0f}')
model.summary()

Flops per frame:5790
Model: "NoobHeart"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_normalization_41 (Laye (None, 32, 8, 8, 3)       64        
_________________________________________________________________
conv3d_39 (Conv3D)           (None, 32, 4, 4, 4)       100       
_________________________________________________________________
layer_normalization_42 (Laye (None, 32, 4, 4, 4)       64        
_________________________________________________________________
conv3d_40 (Conv3D)           (None, 32, 2, 2, 2)       66        
_________________________________________________________________
layer_normalization_43 (Laye (None, 32, 2, 2, 2)       64        
_________________________________________________________________
average_pooling3d_13 (Averag (None, 32, 1, 1, 2)       0         
_________________________________________________________________
conv3d_41 (Conv3D)           (None, 

In [57]:
model.fit(train.shuffle(9999).batch(32), validation_data=valid.batch(32), epochs=5, callbacks=[keras.callbacks.ModelCheckpoint('../weights/NoobHeart_SCAMPS.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
model.load_weights('../weights/NoobHeart_SCAMPS.h5')

Epoch 1/5
1185/1185 [==============================] - 3s 2ms/step - loss: 0.7987 - val_loss: 0.7144

Epoch 00001: val_loss improved from inf to 0.71439, saving model to ../weights\NoobHeart_SCAMPS.h5
Epoch 2/5
1185/1185 [==============================] - 2s 2ms/step - loss: 0.7633 - val_loss: 0.6847

Epoch 00002: val_loss improved from 0.71439 to 0.68467, saving model to ../weights\NoobHeart_SCAMPS.h5
Epoch 3/5
1185/1185 [==============================] - 2s 2ms/step - loss: 0.7512 - val_loss: 0.6664

Epoch 00003: val_loss improved from 0.68467 to 0.66635, saving model to ../weights\NoobHeart_SCAMPS.h5
Epoch 4/5
1185/1185 [==============================] - 2s 2ms/step - loss: 0.7397 - val_loss: 0.6618

Epoch 00004: val_loss improved from 0.66635 to 0.66183, saving model to ../weights\NoobHeart_SCAMPS.h5
Epoch 5/5
1185/1185 [==============================] - 2s 2ms/step - loss: 0.7286 - val_loss: 0.6625

Epoch 00005: val_loss did not improve from 0.66183


In [17]:
model.load_weights('../weights/NoobHeart.h5')

In [76]:
eval_on_dataset(test_set_CCNU, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_SCAMPS_CCNU.h5')
get_metrics('../results/NoobHeart_SCAMPS_CCNU.h5',)

100%|██████████| 179/179 [00:14<00:00, 12.40it/s]


{'Sliding window': {'MAE': 1.784, 'RMSE': 5.851, 'R': 0.8307},
 'Whole video': {'MAE': 0.961, 'RMSE': 2.933, 'R': 0.95338}}

In [15]:
eval_on_dataset(test_set_CCNU_rPPG, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_SCAMPS_CCNU_rPPG.h5')
get_metrics('../results/NoobHeart_SCAMPS_CCNU_rPPG.h5',)

100%|██████████| 54/54 [00:03<00:00, 15.05it/s]


{'Sliding window': {'MAE': 2.075, 'RMSE': 5.693, 'R': 0.83425},
 'Whole video': {'MAE': 0.794, 'RMSE': 1.031, 'R': 0.99644}}

In [59]:
eval_on_dataset(test_set_PURE, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_SCAMPS_PURE.h5')
get_metrics('../results/NoobHeart_SCAMPS_PURE.h5',)

100%|██████████| 59/59 [00:01<00:00, 33.19it/s]


{'Sliding window': {'MAE': 1.029, 'RMSE': 5.574, 'R': 0.97025},
 'Whole video': {'MAE': 0.53, 'RMSE': 0.876, 'R': 0.99935}}

In [58]:
eval_on_dataset(test_set_UBFC_rPPG2, model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_SCAMPS_UBFC.h5')
get_metrics('../results/NoobHeart_SCAMPS_UBFC.h5',)

100%|██████████| 42/42 [00:01<00:00, 34.31it/s]


{'Sliding window': {'MAE': 1.713, 'RMSE': 4.615, 'R': 0.96497},
 'Whole video': {'MAE': 1.046, 'RMSE': 1.492, 'R': 0.9966}}